In [1]:
import numpy as np
import pandas as pd
from numba import jit

In [4]:
## Load data

mdata = np.array(pd.read_csv('DataCounterfactual.csv',header=None))

# Reformat data

vmgrno_in       = np.array(mdata[:,0]).reshape(-1,1)
vpermno_in      = np.array(mdata[:,1]).reshape(-1,1)

# Organize data - rows: stocks; columns: managers

vmgrno          = np.unique(vmgrno_in).reshape(-1,1)
vpermno         = np.unique(vpermno_in).reshape(-1,1)

iNmgr           = len(vmgrno)
iNstocks        = len(vpermno)

mlatent         = np.zeros((iNstocks,iNmgr))
mweight         = np.zeros((iNstocks,iNmgr))

vLNme           = np.zeros((iNstocks,1))
vLNme[:,0]      = np.NaN

vaum            = np.zeros((iNmgr,1))
voutweight      = np.zeros((iNmgr,1))
vbME            = np.zeros((iNmgr,1))
vrank_mgr       = np.zeros((iNmgr,1))  
vaum[:,0]       = np.NaN
voutweight[:,0] = np.NaN
vbME[:,0]       = np.NaN
vrank_mgr[:,0]  = np.NaN

for i in range(iNmgr):

    vsel             = np.where(vmgrno_in[:,0] == vmgrno[i,0])[0]

    vpermno_in_sel   = np.array(mdata[vsel,1]).reshape(-1,1)

    vaum[i,0]        = np.array(mdata[vsel[0],3])
    voutweight[i,0]  = np.array(mdata[vsel[0],5])
    vbME[i,0]        = np.array(mdata[vsel[0],6])

    vweight_in       = np.array(mdata[vsel,4]).reshape(-1,1)
    vlatent_in       = np.array(mdata[vsel,7]).reshape(-1,1)


    for n in range(iNstocks):

        isel         = np.where(vpermno_in_sel[:,0] == vpermno[n,0])[0]

        if len(isel) != 0:

            mweight[n,i]  = vweight_in[isel,0]
            mlatent[n,i]  = vlatent_in[isel,0]

for n in range(iNstocks):

    vsel                  = np.where(vpermno_in[:,0] == vpermno[n,0])[0]
    vLNme[n]              = np.array(mdata[vsel[0],2])
    

In [7]:
## Counterfactuals

mLNme_cf      = np.zeros((iNstocks,iNmgr))
mLNme_cf[:,:] = np.NaN

for i in range(1,iNmgr):

    print((i+1)/iNmgr)
    
    # Initialize market caps
    
    vLNmeLOOP        = vLNme
    
    # Set latent demand to zero
    
    mlatentLOOP      = mlatent
    mlatentLOOP[:,i] = np.zeros(iNstocks)     
    
    # Compute the flow to other managers (adjust for HH)
    
    iHH              = np.where(vmgrno[:,0] == 0)[0]
    daumHH           = vaum[iHH]
        
    vS               = vaum / (np.sum(vaum) - vaum[i] - daumHH)
    vS[i]            = 0
    vS[iHH]          = 0
    vflow            = vaum[i] * vS
    
    # Main loop: Solve for prices
    
    istep   = 1
    dgap    = 1
    
    while istep < 1e3 and dgap > 1e-4:
        
        # update AUM
        
        vaumLOOP      = vaum * (mweight.T @ np.exp(vLNmeLOOP - vLNme) + voutweight) + vflow
        
        # Update demand
        
        mweightD      = mlatentLOOP * np.exp(vLNmeLOOP @ vbME.T)
        mweightLOOP   = mweightD / (np.ones((iNstocks,1)) @ (1 + np.sum(mweightD,0,keepdims=True)))
        
        vdemand       = np.sum(mweightLOOP * (np.ones((iNstocks,1)) @ vaumLOOP.T),1,keepdims=True)
        
        vDdemand      = np.sum((np.ones((iNstocks,1)) @ (vbME * vaumLOOP).T) \
                             * mweightLOOP * (1 - mweightLOOP) / (vdemand @ np.ones((1,iNmgr))),1,keepdims=True)
        
        temp          = vDdemand.copy()
        temp[temp>0]  = 0
        vDemand       = 1./(1 - temp)
        
        # Check convergence
        
        vgap          = np.log(vdemand) - vLNmeLOOP
        dgap          = np.max(np.abs(vgap))
                
        # Update price
        
        vLNmeLOOP     = vLNmeLOOP + vDdemand * vgap
        istep         = istep + 1
        
    
    # Store the data
    
    mLNme_cf[:,i]     = vLNmeLOOP.reshape(-1)
    

In [ ]:
## Counterfactuals

mLNme_cf      = np.zeros((iNstocks,iNmgr))
mLNme_cf[:,:] = np.NaN

for i in range(1,iNmgr):

    print((i+1)/iNmgr)
    
    # Initialize market caps
    
    vLNmeLOOP        = vLNme
    
    # Set latent demand to zero
    
    mlatentLOOP      = mlatent
    mlatentLOOP[:,i] = np.zeros(iNstocks)     
    
    # Compute the flow to other managers (adjust for HH)
    
    iHH              = np.where(vmgrno[:,0] == 0)[0]
    daumHH           = vaum[iHH]
        
    vS               = vaum / (np.sum(vaum) - vaum[i] - daumHH)
    vS[i]            = 0
    vS[iHH]          = 0
    vflow            = vaum[i] * vS
    
    # Main loop: Solve for prices
    
    istep   = 1
    dgap    = 1
    
    while istep < 1e3 and dgap > 1e-4:
        
        # update AUM
        
        vaumLOOP      = vaum * (mweight.T @ np.exp(vLNmeLOOP - vLNme) + voutweight) + vflow
        
        # Update demand
        
        mweightD      = mlatentLOOP * np.exp(vLNmeLOOP @ vbME.T)
        mweightLOOP   = mweightD / (np.ones((iNstocks,1)) @ (1 + np.sum(mweightD,0,keepdims=True)))
        
        vdemand       = np.sum(mweightLOOP * (np.ones((iNstocks,1)) @ vaumLOOP.T),1,keepdims=True)
        
        vDdemand      = np.sum((np.ones((iNstocks,1)) @ (vbME * vaumLOOP).T) \
                             * mweightLOOP * (1 - mweightLOOP) / (vdemand @ np.ones((1,iNmgr))),1,keepdims=True)
        
        temp          = vDdemand.copy()
        temp[temp>0]  = 0
        vDemand       = 1./(1 - temp)
        
        # Check convergence
        
        vgap          = np.log(vdemand) - vLNmeLOOP
        dgap          = np.max(np.abs(vgap))
                
        # Update price
        
        vLNmeLOOP     = vLNmeLOOP + vDdemand * vgap
        istep         = istep + 1
        
    
    # Store the data
    
    mLNme_cf[:,i]     = vLNmeLOOP.reshape(-1)
    